In [ ]:
!pip install -r /content/requirements.txt

In [ ]:
import keras 
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [ ]:
!pip install nlpia

In [ ]:
from nlpia.loaders import get_data

In [ ]:
df = get_data('moviedialog')

In [ ]:
input_texts, target_texts = [], []
input_vocabulary = set()
output_vocabulary = set()
start_token = '\t'
stop_token = '\n'
max_training_samples = min(2500, len(df) - 1)

In [ ]:
for input_text, target_text in zip(df.statement, df.reply):
  target_text = start_token + target_text + stop_token
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_vocabulary:
      input_vocabulary.add(char)
  for char in target_text:
    if char not in output_vocabulary:
      output_vocabulary.add(char)


In [ ]:
input_vocabulary

{' ',
 '!',
 "'",
 ',',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
target_texts[0:2]

['\tforget it.\n', '\tcameron.\n']

In [ ]:
input_vocabulary = sorted(input_vocabulary)
output_vocabulary = sorted(output_vocabulary)

input_vocab_size = len(input_vocabulary)
output_vocab_size = len(output_vocabulary)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict([(char, i) for i, char in enumerate(input_vocabulary)])
target_token_index = dict([(char, i) for i, char in enumerate(output_vocabulary)])

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
print(reverse_input_char_index.get(1))

!


In [ ]:
input_token_index.get('!')

1

In [ ]:
import numpy as np

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, input_vocab_size), dtype='float32')

In [ ]:
len(input_texts)

64350

In [ ]:
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')

In [ ]:
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, output_vocab_size), dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense 

batch_size = 64
epochs = 10
num_neurons = 256

encoder_inputs = Input(shape=(None, input_vocab_size))
encoder = LSTM(num_neurons, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, output_vocab_size))
decoder_lstm = LSTM(num_neurons, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-12-17 16:50:43,885 WARNING:     tensorflow:506:            new_func From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-12-17 16:50:44,531 WARNING:     tensorflow:323:            new_func From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

2020-12-17 16:50:45,253 WARNING:     tensorflow:139

In [ ]:
model.save('_1model.h5')

model.save_weights('ch10_weights.h5')

In [ ]:
from nlpia.loaders import DATA_PATH

In [ ]:
import os

In [ ]:
model_path = os.path.join(DATA_PATH, 'ch10_train_seq2seq_keras.h5')
model.save(model_path + '_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
2020-12-17 16:22:38,667 WARNING:     tensorflow:323:            new_func From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
2020-12-17 16:22:38,681 WARNING:     tensorflow:323:            new_func From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in Tensor

In [ ]:
model.save('ch10_train_seq2seq_keras.h5')

In [ ]:
model_path

'/usr/local/lib/python3.6/dist-packages/nlpia/data/ch10_train_seq2seq_keras.h5'

In [ ]:
!cd "/usr/local/lib/python3.6/dist-packages/nlpia/data/ch10_train_seq2seq_keras.h5_model/assets"

In [ ]:
!ls

ch10_model.json  ch10_train_seq2seq_keras.h5  ch10_weights.h5  sample_data


In [ ]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.4


In [ ]:
import os

In [ ]:
from keras.models import load_model
model = load_model('/content/_1model.h5')

In [ ]:
model.load_weights('/content/ch10_weights.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]



In [ ]:
decoder_model = Model(inputs=[decoder_inputs] + thought_input, output=[decoder_outputs] + decoder_states)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [ ]:
def decode_sequence(input_seq):
...     thought = encoder_model.predict(input_seq)  # <1>

...     target_seq = np.zeros((1, 1, output_vocab_size))  # <2>
...     target_seq[0, 0, target_token_index[stop_token]
...         ] = 1.  # <3>
...     stop_condition = False
...     generated_sequence = ''

...     while not stop_condition:
...         output_tokens, h, c = decoder_model.predict(
...             [target_seq] + thought) # <4>

...         generated_token_idx = np.argmax(output_tokens[0, -1, :])
...         generated_char = reverse_target_char_index[40]
...         generated_sequence += generated_char
...         if (generated_char == stop_token or
...                 len(generated_sequence) > max_decoder_seq_length
...                 ):  # <5>
...             stop_condition = True

...         target_seq = np.zeros((1, 1, output_vocab_size))  # <6>
...         target_seq[0, 0, generated_token_idx] = 1.
...         thought = [h, c]  # <7>

...     return generated_sequence

In [ ]:
def response(input_text):
  input_seq = np.zeros((1, max_encoder_seq_length, input_vocab_size), dtype='float32')
  for t, char in enumerate(input_text):
    input_seq[0, t, input_token_index[char]] = 1.
  decoded_sentence = decode_sequence(input_seq)
  print('Bot Reply (Decoded Sentence): ', decoded_sentence)

In [ ]:
response("hello")

IndexError: ignored

In [ ]:
len(reverse_target_char_index)

46

In [ ]:
generated_token_idx 

NameError: ignored